# Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import os
import numpy as np


# Functions

In [3]:
def front_fill_0 (list, x):
    new_list=[]
    for id in list:
        if len(id)<x:
            id = '0'*(8-len(id))+id
        new_list.append(id)
    return new_list

def contains_letters(string):
    for char in string:
        if char.isalpha():
            return True
    return False

# Basic Data

In [270]:
'''
Building basic database:
takes in data from handcopied file of browser transfers
turns it into DataFrame
'''

alp_path='Data from alp.cz/Data from alp.cz.txt'

with open(alp_path,'r', encoding='utf8') as f:
    lines = f.readlines()
num=0
all_dicts=[]
for line in lines:
    num+=1
    # print(num)
    if num in range(10):
        start = line[61:-3][:20]
        end = line[61:-3][-20:]
        dicts=list(eval(line[61:-3]))
    else:
        start = line[62:-3][:20]
        end = line[61:-3][-20:]
        dicts=list(eval(line[62:-3]))
    # print(start, '  ', end)
    all_dicts = all_dicts + dicts
basic_data = pd.DataFrame(all_dicts)
basic_data.drop_duplicates(subset=['ico'], inplace=True)
basic_data.rename(columns={'ico':'ICO'}, inplace=True)
basic_data

,ICO,name,established,addressCity,dissolved
0,64049761,""" Bytové družstvo Letná 229""","Jul 15, 1996 12:00:00 AM",Mimoň,NaN
1,25198033,""" Bytové družstvo Nerudova 51 ""","Aug 9, 1999 12:00:00 AM",České Budějovice,NaN
2,00483010,""" Bytové družstvo Šanov "" v likvidaci","May 26, 1990 12:00:00 AM",Teplice,"Apr 18, 2019 12:00:00 AM"
3,25167278,""" Bytové družstvo Vltava 12 ""","Sep 26, 1997 12:00:00 AM",České Budějovice,NaN
4,26921065,""" KAMÍNKY - 1, bytové družstvo ""","Mar 3, 2004 12:00:00 AM",Rosice,NaN
...,...,...,...,...,...
27515,61464660,"1538, Bytové družstvo nájemníků","Aug 11, 1994 12:00:00 AM",Praha,NaN
27516,00033839,"2. Stavební bytové družstvo občanů v Praze 4, ...","Jan 1, 1972 12:00:00 AM",Praha,NaN
27517,26891361,"32 BJ Pražská, bytové družstvo","Apr 25, 2003 12:00:00 AM",Znojmo,NaN
27518,64653153,"""641 - BYTOVÉ DRUŽSTVO OSEK""","Feb 19, 1996 12:00:00 AM",Osek,NaN


# Detail Data

In [271]:
'''
Prepares for merger of batch files
'''

path = 'Data from alp.cz/detail_batches'
all_files = os.listdir(path)
data_files = [file for file in all_files if 'data' in file]
dead_files = [file for file in all_files if 'dead' in file]


columns_10 =['ICO',
        'Statistická právní forma',
        'Datum vzniku',
        'Adresa sídla',
        'Okres (CZ-NUTS)',
        'Obec',
        'Hlavní ekonomická činnost (CZ NACE)',
        'Ostatní ekonomické činnosti (CZ NACE)',
        'Institucionální sektor (ESA 2010)',
        'Velikostní kat. dle počtu zaměstnanců',]

columns_12=['ICO',
        'Statistická právní forma',
        'Datum vzniku',
        'Datum zániku',
        'Způsob zániku',
        'Adresa sídla',
        'Okres (CZ-NUTS)',
        'Obec',
        'Hlavní ekonomická činnost (CZ NACE)',
        'Ostatní ekonomické činnosti (CZ NACE)',
        'Institucionální sektor (ESA 2010)',
        'Velikostní kat. dle počtu zaměstnanců']

In [272]:
'''
Merges living entities
'''
data = pd.DataFrame()
counter = 0
for file in data_files:
    counter+=1
    temp = pd.read_csv(path + '/' + file)
    data = pd.concat([data, temp])
data.columns = columns_10
data
data[['Datum zániku','Způsob zániku']]=np.nan
data = data[columns_12]
# data

,ICO,Statistická právní forma,Datum vzniku,Datum zániku,Způsob zániku,Adresa sídla,Okres (CZ-NUTS),Obec,Hlavní ekonomická činnost (CZ NACE),Ostatní ekonomické činnosti (CZ NACE),Institucionální sektor (ESA 2010),Velikostní kat. dle počtu zaměstnanců
0,64049761,205 - Družstvo,1996-07-15 00:00:00.0,NaN,NaN,"Mimoň, 47124, Mimoň IV, Letná 229",CZ0511 - Česká Lípa,561835 - Mimoň,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
1,25198033,205 - Družstvo,1999-08-09 00:00:00.0,NaN,NaN,"České Budějovice, 37004, České Budějovice 3, N...",CZ0311 - České Budějovice,544256 - České Budějovice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
2,25167278,205 - Družstvo,1997-09-26 00:00:00.0,NaN,NaN,"České Budějovice, 37011, České Budějovice 2, B...",CZ0311 - České Budějovice,544256 - České Budějovice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
3,26921065,205 - Družstvo,2004-03-03 00:00:00.0,NaN,NaN,"Rosice, 66501, Rosice, Lipová 1316",CZ0643 - Brno-venkov,583782 - Rosice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
4,26330075,205 - Družstvo,2001-07-11 00:00:00.0,NaN,NaN,"Cheb, 35002, Cheb, Družstevní 1484/7",CZ0411 - Cheb,554481 - Cheb,6820 - Pronájem a správa vlastních nebo pronaj...,"\n257 - Výroba nožířských výrobků, nástrojů a ...",11002 - Národní soukromé nefinanční podniky,Neuvedeno
...,...,...,...,...,...,...,...,...,...,...,...,...
25,28644654,205 - Družstvo,2011-01-06 00:00:00.0,NaN,NaN,"Ostrava, 70030, Výškovice, Lumírova 523/28",CZ0806 - Ostrava-město,546135 - Ostrava-Jih,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
26,26821354,205 - Družstvo,2003-12-05 00:00:00.0,NaN,NaN,"Ostrava, 70030, Výškovice, Lumírova 536/54",CZ0806 - Ostrava-město,546135 - Ostrava-Jih,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
27,27761886,205 - Družstvo,2006-03-21 00:00:00.0,NaN,NaN,"Ostrava, 70030, Výškovice, Lumírova 538/60",CZ0806 - Ostrava-město,546135 - Ostrava-Jih,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
28,26861135,205 - Družstvo,2005-05-26 00:00:00.0,NaN,NaN,"Ostrava, 70030, Výškovice, Lumírova 539/62",CZ0806 - Ostrava-město,546135 - Ostrava-Jih,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno


In [273]:
'''
Merges dead entities
'''

dead = pd.DataFrame()
counter = 0
for file in dead_files:
    counter+=1
    try:
        temp = pd.read_csv(path + '/' + file)
        dead = pd.concat([dead, temp])
    except:
        pass
dead.columns = columns_12
# dead

,ICO,Statistická právní forma,Datum vzniku,Datum zániku,Způsob zániku,Adresa sídla,Okres (CZ-NUTS),Obec,Hlavní ekonomická činnost (CZ NACE),Ostatní ekonomické činnosti (CZ NACE),Institucionální sektor (ESA 2010),Velikostní kat. dle počtu zaměstnanců
0,483010,205 - Družstvo,1990-05-26 00:00:00.0,2019-04-18 00:00:00.0,04 - Zrušení právnické osoby bez likvidace bez...,"Teplice, 41501, Teplice, Josefa Šafaříka 1596/3",CZ0426 - Teplice,567442 - Teplice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
1,26901935,205 - Družstvo,2003-07-24 00:00:00.0,2021-06-04 00:00:00.0,01 - Zrušení právnických osob likvidací,"Brno, Rostislavovo náměstí 5, PSČ 61200",CZ0642 - Brno-město,551007 - Brno-Královo Pole,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
2,26292653,205 - Družstvo,2002-08-21 00:00:00.0,2021-12-07 00:00:00.0,01 - Zrušení právnických osob likvidací,"Brno, 61500, Židenice (Brno-Židenice), Kosmáko...",CZ0642 - Brno-město,551058 - Brno-Židenice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
3,506176,205 - Družstvo,1990-04-05 00:00:00.0,2021-08-17 00:00:00.0,01 - Zrušení právnických osob likvidací,"Praha, 14000, Nusle (Praha 4), U gymnázia 126/5",CZ0100 - Praha,500119 - Praha 4,68320 - Správa nemovitostí na základě smlouvy ...,\n6820 - Pronájem a správa vlastních nebo pron...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
4,530590,205 - Družstvo,1990-05-08 00:00:00.0,2022-06-01 00:00:00.0,01 - Zrušení právnických osob likvidací,Slušovice,CZ0724 - Zlín,585777 - Slušovice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
...,...,...,...,...,...,...,...,...,...,...,...,...
4,61863424,205 - Družstvo,1994-10-20 00:00:00.0,2020-08-04 00:00:00.0,01 - Zrušení právnických osob likvidací,"Praha, 14200, Kamýk, Liškova 632/6",CZ0100 - Praha,547107 - Praha 12,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
0,26837510,205 - Družstvo,2004-06-30 00:00:00.0,2020-05-05 00:00:00.0,01 - Zrušení právnických osob likvidací,"Šternberk, 78501, Šternberk, Lomená 1445/13",CZ0712 - Olomouc,505188 - Šternberk,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
1,3083632,205 - Družstvo,2014-06-06 00:00:00.0,2022-04-29 00:00:00.0,01 - Zrušení právnických osob likvidací,"Praha, 12000, Vinohrady (Praha 2), Londýnská 8...",CZ0100 - Praha,500089 - Praha 2,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
2,63468859,205 - Družstvo,1995-07-10 00:00:00.0,2021-02-11 00:00:00.0,01 - Zrušení právnických osob likvidací,"Brno, 61400, Husovice, Lozíbky 1478/58",CZ0642 - Brno-město,551031 - Brno-sever,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno


# Meagring

In [407]:
'''
Merges living and dead entities
'''
detail_combined = pd.concat([data,dead])
detail_combined.ICO = front_fill_0(detail_combined.ICO.astype(str), 8)
'''
Merges basic and detail data
'''
CZ_database = detail_combined.merge(basic_data[['ICO','name']], how='left', on='ICO')
CZ_database = CZ_database[columns_12[:1]+['name']+columns_12[1:]]
CZ_database.to_csv('Data from alp.cz/CZ_database_dirty.csv', index=False )

CZ_database

,ICO,name,Statistická právní forma,Datum vzniku,Datum zániku,Způsob zániku,Adresa sídla,Okres (CZ-NUTS),Obec,Hlavní ekonomická činnost (CZ NACE),Ostatní ekonomické činnosti (CZ NACE),Institucionální sektor (ESA 2010),Velikostní kat. dle počtu zaměstnanců
0,64049761,""" Bytové družstvo Letná 229""",205 - Družstvo,1996-07-15 00:00:00.0,NaN,NaN,"Mimoň, 47124, Mimoň IV, Letná 229",CZ0511 - Česká Lípa,561835 - Mimoň,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
1,25198033,""" Bytové družstvo Nerudova 51 """,205 - Družstvo,1999-08-09 00:00:00.0,NaN,NaN,"České Budějovice, 37004, České Budějovice 3, N...",CZ0311 - České Budějovice,544256 - České Budějovice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
2,25167278,""" Bytové družstvo Vltava 12 """,205 - Družstvo,1997-09-26 00:00:00.0,NaN,NaN,"České Budějovice, 37011, České Budějovice 2, B...",CZ0311 - České Budějovice,544256 - České Budějovice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
3,26921065,""" KAMÍNKY - 1, bytové družstvo """,205 - Družstvo,2004-03-03 00:00:00.0,NaN,NaN,"Rosice, 66501, Rosice, Lipová 1316",CZ0643 - Brno-venkov,583782 - Rosice,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
4,26330075,""" Skalka - Cheb, bytové družstvo """,205 - Družstvo,2001-07-11 00:00:00.0,NaN,NaN,"Cheb, 35002, Cheb, Družstevní 1484/7",CZ0411 - Cheb,554481 - Cheb,6820 - Pronájem a správa vlastních nebo pronaj...,"\n257 - Výroba nožířských výrobků, nástrojů a ...",11002 - Národní soukromé nefinanční podniky,Neuvedeno
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8366,61863424,"Bytové družstvo Liškova 632/633, v likvidaci",205 - Družstvo,1994-10-20 00:00:00.0,2020-08-04 00:00:00.0,01 - Zrušení právnických osob likvidací,"Praha, 14200, Kamýk, Liškova 632/6",CZ0100 - Praha,547107 - Praha 12,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
8367,26837510,"Bytové družstvo Lomená 13, v likvidaci",205 - Družstvo,2004-06-30 00:00:00.0,2020-05-05 00:00:00.0,01 - Zrušení právnických osob likvidací,"Šternberk, 78501, Šternberk, Lomená 1445/13",CZ0712 - Olomouc,505188 - Šternberk,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
8368,03083632,"Bytové družstvo Londýnská č.p. 815, Praha 2 v ...",205 - Družstvo,2014-06-06 00:00:00.0,2022-04-29 00:00:00.0,01 - Zrušení právnických osob likvidací,"Praha, 12000, Vinohrady (Praha 2), Londýnská 8...",CZ0100 - Praha,500089 - Praha 2,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno
8369,63468859,"Bytové družstvo Lozíbky 52,54,56,58, družstvo ...",205 - Družstvo,1995-07-10 00:00:00.0,2021-02-11 00:00:00.0,01 - Zrušení právnických osob likvidací,"Brno, 61400, Husovice, Lozíbky 1478/58",CZ0642 - Brno-město,551031 - Brno-sever,6820 - Pronájem a správa vlastních nebo pronaj...,\r\n -\r\n ...,11002 - Národní soukromé nefinanční podniky,Neuvedeno


# Cleaning

In [4]:
def remove_mult_front_back(list, ers ,front=True, back=True):
    new_list=[]
    for i in list:
        # print(i)
        for er in ers:
            if front:
                if i[:len(er)]==er:
                    i = i[len(er):]
                    # print(i)
            if back:
                if i[-len(er):]==er:
                    i = i[:-len(er)]
                    # print(i)
        new_list.append(i)
    return new_list

In [ ]:
CZ_database = pd.read_csv('Data from alp.cz/CZ_database_dirty.csv')

CZ_database.name = remove_mult_front_back(CZ_database.name, ['"',"'",' '])


CZ_database['Datum vzniku'] = [f[:-11] for f in CZ_database['Datum vzniku']]

CZ_database['Ostatní ekonomické činnosti (CZ NACE)'] = remove_mult_front_back(CZ_database['Ostatní ekonomické činnosti (CZ NACE)'], '\n')
CZ_database['Ostatní ekonomické činnosti (CZ NACE)'].replace('\r\n         ', np.nan, regex=True, inplace=True)
CZ_database.to_csv('CZ_database.csv', index=False )
CZ_database.to_excel('CZ_database.xlsx', index=False )
CZ_database